# Field Line Detection on Football Pitch

This notebook demonstrates a traditional image-processing pipeline for detecting field lines on a football pitch. It includes multiple alternative algorithms at each step so you can select the best one for your application. The pipeline consists of:

1. **Crowd Masking:** Use the predominance of green on the pitch to mask out non-pitch areas.
   - Option A: HSV thresholding
   - Option B: LAB thresholding

2. **White Line Extraction:** Isolate the white markings by thresholding the masked pitch image.
   - Option A: Simple binary thresholding
   - Option B: Adaptive thresholding

3. **Edge Detection:** Extract edges using:
   - Option A: Canny edge detector
   - Option B: Sobel operator

4. **Line Detection:** Detect line segments using:
   - Option A: Probabilistic Hough Transform (HoughLinesP)
   - Option B: Line Segment Detector (LSD)

Each cell displays intermediate output along with debugging information. Adjust the algorithm selection variables as needed.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys

def show_image(img, title='Image', cmap=None):
    plt.figure(figsize=(10, 6))
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        # Convert BGR to RGB
        if len(img.shape) == 3:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

def debug_print(msg):
    print("[DEBUG] " + msg, file=sys.stderr)

debug_print("Libraries imported successfully.")

In [ ]:
# Load the input image
image_path = 'input.jpg'
image = cv2.imread(image_path)

if image is None:
    debug_print(f"Error: Could not load image '{image_path}'.")
    sys.exit(1)
else:
    debug_print(f"Image '{image_path}' loaded. Shape: {image.shape}")
    show_image(image, 'Original Image')

## Step 1: Crowd Masking

We mask out non-pitch areas by taking advantage of the pitch's predominant green color. Two methods are provided:

- **HSV Thresholding:** Convert to HSV and threshold on a green range.
- **LAB Thresholding:** Convert to LAB and threshold on the `a` channel to detect green areas.

Set `mask_method` to `'HSV'` or `'LAB'` to choose the algorithm.

In [ ]:
# Choose masking method: 'HSV' or 'LAB'
mask_method = 'HSV'  # Change to 'LAB' to try LAB thresholding

debug_print(f"Using {mask_method} thresholding for pitch masking.")

if mask_method == 'HSV':
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    debug_print("Converted image to HSV.")
    
    # Define green range in HSV
    lower_green = np.array([25, 20, 20])
    upper_green = np.array([95, 255, 255])
    pitch_mask = cv2.inRange(hsv, lower_green, upper_green)
    debug_print("HSV mask created.")
    
elif mask_method == 'LAB':
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    debug_print("Converted image to LAB.")
    
    # In LAB, green usually has lower 'a' values. These values may need tuning.
    lower_lab = np.array([0, 110, 0])
    upper_lab = np.array([255, 140, 255])
    pitch_mask = cv2.inRange(lab, lower_lab, upper_lab)
    debug_print("LAB mask created.")
else:
    debug_print("Invalid mask method selected. Defaulting to HSV.")
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 20, 20])
    upper_green = np.array([95, 255, 255])
    pitch_mask = cv2.inRange(hsv, lower_green, upper_green)

# Apply morphological operations to smooth the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
pitch_mask = cv2.morphologyEx(pitch_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
pitch_mask = cv2.dilate(pitch_mask, kernel, iterations=1)
debug_print("Morphological operations applied to mask.")

show_image(pitch_mask, 'Pitch Mask', cmap='gray')

# Extract pitch region (mask out crowd)
pitch_only = cv2.bitwise_and(image, image, mask=pitch_mask)
debug_print("Extracted pitch region using the mask.")
show_image(pitch_only, 'Pitch Only (Crowd Masked Out)')

## Step 2: White Line Extraction

Here we isolate the white field markings from the pitch region. Two approaches are provided:

- **Simple Binary Thresholding**: A fixed threshold is used.
- **Adaptive Thresholding**: The threshold is computed locally for better performance in uneven lighting.

Set `thresh_method` to `'simple'` or `'adaptive'`.

In [ ]:
# Choose thresholding method: 'simple' or 'adaptive'
thresh_method = 'simple'  # Change to 'adaptive' to try adaptive thresholding
debug_print(f"Using {thresh_method} thresholding for white line extraction.")

# Convert pitch-only image to grayscale
gray = cv2.cvtColor(pitch_only, cv2.COLOR_BGR2GRAY)
show_image(gray, 'Grayscale Pitch Image', cmap='gray')

if thresh_method == 'simple':
    # Use a fixed threshold (tuned for white lines)
    ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    debug_print(f"Simple threshold applied. ret = {ret}")
elif thresh_method == 'adaptive':
    # Use adaptive thresholding for better handling of varying illumination
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                   cv2.THRESH_BINARY, blockSize=11, C=2)
    debug_print("Adaptive threshold applied.")
else:
    debug_print("Invalid threshold method selected. Defaulting to simple thresholding.")
    ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

show_image(thresh, 'Thresholded Image (White Lines Highlighted)', cmap='gray')

## Step 3: Edge Detection

Next, we extract edges from the thresholded image. Two options are provided:

- **Canny Edge Detector**
- **Sobel Operator**

Set `edge_method` to `'canny'` or `'sobel'`.

In [ ]:
# Choose edge detection method: 'canny' or 'sobel'
edge_method = 'canny'  # Change to 'sobel' to try Sobel operator
debug_print(f"Using {edge_method} edge detection.")

if edge_method == 'canny':
    edges = cv2.Canny(thresh, 50, 150, apertureSize=3)
    debug_print("Canny edge detection applied.")
elif edge_method == 'sobel':
    # Apply Sobel operator in both x and y directions
    sobelx = cv2.Sobel(thresh, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(thresh, cv2.CV_64F, 0, 1, ksize=3)
    edges = cv2.magnitude(sobelx, sobely).astype(np.uint8)
    debug_print("Sobel edge detection applied.")
else:
    debug_print("Invalid edge method selected. Defaulting to Canny.")
    edges = cv2.Canny(thresh, 50, 150, apertureSize=3)

show_image(edges, 'Edge Map', cmap='gray')

## Step 4: Line Detection

Finally, we detect line segments from the edge map. Two alternatives are provided:

- **HoughLinesP (Probabilistic Hough Transform)**
- **Line Segment Detector (LSD)**

Set `line_method` to `'hough'` or `'lsd'`.

In [ ]:
# Choose line detection method: 'hough' or 'lsd'
line_method = 'hough'  # Change to 'lsd' to try Line Segment Detector
debug_print(f"Using {line_method} for line detection.")

# Create a copy of the pitch-only image to draw detected lines
line_image = pitch_only.copy()

if line_method == 'hough':
    # Use Probabilistic Hough Transform
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, 
                            minLineLength=50, maxLineGap=10)
    if lines is not None:
        debug_print(f"HoughLinesP detected {len(lines)} lines.")
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    else:
        debug_print("No lines detected using HoughLinesP.")

elif line_method == 'lsd':
    # Use Line Segment Detector (LSD)
    lsd = cv2.createLineSegmentDetector(0)
    lines_lsd = lsd.detect(edges)[0]  # returns a list of lines
    if lines_lsd is not None:
        debug_print(f"LSD detected {len(lines_lsd)} line segments.")
        for line in lines_lsd:
            x1, y1, x2, y2 = map(int, line[0])
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
        debug_print("No lines detected using LSD.")
else:
    debug_print("Invalid line detection method selected. Defaulting to HoughLinesP.")
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, 
                            minLineLength=50, maxLineGap=10)
    if lines is not None:
        debug_print(f"HoughLinesP detected {len(lines)} lines.")
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

show_image(line_image, 'Detected Lines on Pitch')

## Conclusion

This notebook provided a complete multi-algorithm approach for detecting lines on a football pitch. Experiment with different selections for `mask_method`, `thresh_method`, `edge_method`, and `line_method` to find the best combination for your data. Debug messages are printed to stderr for further inspection of internal state and algorithm performance.